In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
name = []
mileage = []
dealer_name = []
rating = []
rating_count = []
price = []

for i in range(1,41):
    
    # Website in a variable
    website = 'https://www.cars.com/shopping/results/?page=' + str(i) + '&page_size=20&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=bmw&maximum_distance=all&mileage_max=&models[]=bmw-335&sort=best_match_desc&stock_type=used&year_max=2011&year_min=2007&zip='

    # Request to website
    response = requests.get(website)
    
    # Soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Results
    results = soup.find_all('div', {'class':'vehicle-card'})
    
    
    for result in results:
    
        # name
        try:
            name.append(result.find('h2').get_text())
        except:
            name.append('n/a')


        # mileage
        try:
            mileage.append(result.find('div', {'class':'mileage'}).get_text())
        except:
            mileage.append('n/a')

        # dealer_name
        try:
            dealer_name.append(result.find('div', {'class': 'dealer-name'}).get_text().strip())
        except:
            dealer_name.append('n/a')

        # rating
        try:
            rating.append(result.find('span',{'class':'sds-rating__count'}).get_text())
        except:
            rating.append('n/a')


        # rating_count
        try:
            rating_count.append(result.find('span', {'class': 'sds-rating__link'}).get_text())
        except:
            rating_count.append('n/a')


        # price
        try:
            price.append(result.find('span', {'class': 'primary-price'}).get_text())
        except:
            price.append('n/a')

In [5]:
bmw_df = pd.DataFrame({'Name':name, 'Mileage': mileage,'Dealer Name': dealer_name,
                          'Rating': rating, 'Rating Count': rating_count, 'Price':price})

In [6]:
bmw_df

,Name,Mileage,Dealer Name,Rating,Rating Count,Price
0,2009 BMW 335 i,"76,192 mi.",Jet's Auto Sales,3.0,(35 reviews),"$16,995"
1,2011 BMW 335 i xDrive,"66,689 mi.",Fort Wayne Nissan,4.5,(651 reviews),"$15,291"
2,2010 BMW 335 i,"42,951 mi.",Carmel Motors,3.8,(77 reviews),"$21,995"
3,2011 BMW 335 is,"63,000 mi.",n/a,n/a,n/a,"$28,500"
4,2009 BMW 335 i,"80,386 mi.",Mac Haik Chevrolet,4.5,(637 reviews),"$13,991"
...,...,...,...,...,...,...
309,2008 BMW 335 xi,"145,623 mi.",Pro Cars USA Inc,4.3,(13 reviews),"$9,985"
310,2011 BMW 335 i xDrive,"98,463 mi.",Vehicle City Motors,2.3,(7 reviews),"$17,950"
311,2011 BMW 335 i xDrive,"86,645 mi.",M Auto Group,4.8,(6 reviews),"$19,895"
312,2008 BMW 335 i,"135,532 mi.",Volvo Cars Tucson,4.7,(316 reviews),"$8,490"


### Data Cleaning

In [7]:
bmw_df['Rating Count'] = bmw_df['Rating Count'].apply(lambda x: x.strip('reviews)').strip('('))

In [8]:
bmw_df

,Name,Mileage,Dealer Name,Rating,Rating Count,Price
0,2009 BMW 335 i,"76,192 mi.",Jet's Auto Sales,3.0,35,"$16,995"
1,2011 BMW 335 i xDrive,"66,689 mi.",Fort Wayne Nissan,4.5,651,"$15,291"
2,2010 BMW 335 i,"42,951 mi.",Carmel Motors,3.8,77,"$21,995"
3,2011 BMW 335 is,"63,000 mi.",n/a,n/a,n/a,"$28,500"
4,2009 BMW 335 i,"80,386 mi.",Mac Haik Chevrolet,4.5,637,"$13,991"
...,...,...,...,...,...,...
309,2008 BMW 335 xi,"145,623 mi.",Pro Cars USA Inc,4.3,13,"$9,985"
310,2011 BMW 335 i xDrive,"98,463 mi.",Vehicle City Motors,2.3,7,"$17,950"
311,2011 BMW 335 i xDrive,"86,645 mi.",M Auto Group,4.8,6,"$19,895"
312,2008 BMW 335 i,"135,532 mi.",Volvo Cars Tucson,4.7,316,"$8,490"


In [9]:
bmw_df.to_excel('mutiple_pages_car.xlsx', index=False)